# Using ADIOS2 Python with Jupyter Notebook



## Writer: Generate Data
    
    We use a simple solution to the heat transfer equation in a (very-fast) 1D cooling rod.
    

In [1]:
from mpi4py import MPI
import math
import numpy as np
import adios2


def calcUxt(Uo, x, t):
    return 4 * Uo / math.pi * math.sin(math.pi * x) * math.exp(- math.pi * math.pi * t)

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialize dimensions
nx = 10

shape = [size * nx]
start = [rank * nx]
count = [   1 * nx]

# Initialize problem x coord, Uo, Uxt
Lx = 1. 
Dx = 1./shape[0]
x = np.full( (nx), 0. )
for i in range(0,nx):
    x[i] = start[0] + (i+0.5) * Dx

# Make sure you use floats
Uo = 1. 
Uxt = np.full( (nx), Uo)

# Writer
with adios2.open("Uxt.bp", "w", comm) as fw:

    for t in range(0, 5):
        # We save "x" only once
        if(t == 0):
            fw.write("x", x, shape, start, count)
        else:
            for i in range(0,nx):
                Uxt[i] = calcUxt(Uo, x[i], t)
                
        fw.write("Uxt", Uxt, shape, start, count, endl=True)

ModuleNotFoundError: No module named 'adios2'

The data is saved in a bp file and subsfile directory structure. To access the data, the user doesn't need to manipulate these directly, but rather using the **bpls** command line utility for quick inspection. For example:

In [ ]:
!bpls Uxt.bp -lavd -n 10

## Reader: Accessing and Understanding the Data

In [ ]:
from mpi4py import MPI
import math
import numpy as np
import adios2
import matplotlib.pyplot as plt


Uxt_plots = plt.subplots()

with adios2.open("Uxt.bp", "r", comm) as fr:

    for fr_step in fr:

        step = fr_step.currentstep()
        if(step == 0):
            x = fr_step.read("x")
            
        Uxt = fr_step.read("Uxt")
        
        plt.semilogy(x, Uxt, label = "Ux for time= " + str(step))
        
legend = plt.legend(loc='best', shadow=True, fontsize='x-large')